In [1]:
!nvidia-smi

Sun Jun 12 09:06:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/LukaKu/Dataset/Dataset_capstone_final.zip'

In [4]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os 

dir_i = os.listdir('/content/Dataset_img_final')

for i in dir_i:
  dir_a = os.path.join('/content/Dataset_img_final', i)
  print(i, ' has ', len(os.listdir(dir_a)), ' images')

orthopaedic wounds  has  49  images
miscellaneous  has  56  images
leg-ulcer-images  has  134  images
foot-ulcers  has  384  images
pressure ulcer  has  856  images
epidermolysis-bullosa  has  5  images
toes  has  35  images
haemangioma  has  6  images
DFU  has  673  images
burns  has  457  images
malignant-wound-images  has  9  images
meningitis  has  24  images
normal  has  300  images
venous  has  411  images
extravasation-wound-images  has  20  images
pilonidal-sinus  has  3  images
surgical  has  420  images
skin tears  has  54  images
abdominal-wounds  has  13  images
trauma  has  93  images


In [6]:
!mv "/content/Dataset_img_final/pilonidal-sinus" "/content/content"
!mv "/content/Dataset_img_final/haemangioma" "/content/content"
!mv "/content/Dataset_img_final/epidermolysis-bullosa" "/content/content"
!mv "/content/Dataset_img_final/extravasation-wound-images" "/content/content"
!mv "/content/Dataset_img_final/malignant-wound-images" "/content/content"
!mv "/content/Dataset_img_final/abdominal-wounds" "/content/content"
!mv "/content/Dataset_img_final/meningitis" "/content/content"
!mv "/content/Dataset_img_final/toes" "/content/content"
!mv "/content/Dataset_img_final/miscellaneous" "/content/content"
!mv "/content/Dataset_img_final/skin tears" "/content/content"
!mv "/content/Dataset_img_final/trauma" "/content/content"
!mv "/content/Dataset_img_final/orthopaedic wounds" "/content/content"
!mv "/content/Dataset_img_final/leg-ulcer-images" "/content/content"
!mv "/content/Dataset_img_final/normal" "/content/content"
!mv "/content/Dataset_img_final/DFU" "/content/content"

In [7]:
import splitfolders

folder = '/content/Dataset_img_final'

splitfolders.ratio(folder, output='/content/Dataset_split',
                 seed=42, ratio=(0.8, 0.2),
                 group_prefix = None
                 )

Copying files: 2528 files [00:00, 2820.75 files/s]


In [8]:
import os 

dir_i = os.listdir('/content/Dataset_split/train')

for i in dir_i:
  dir_a = os.path.join('/content/Dataset_split/train', i)
  print(i, ' has ', len(os.listdir(dir_a)), ' images')

foot-ulcers  has  307  images
pressure ulcer  has  684  images
burns  has  365  images
venous  has  328  images
surgical  has  336  images


In [9]:
import os 

dir_i = os.listdir('/content/Dataset_split/val')

for i in dir_i:
  dir_a = os.path.join('/content/Dataset_split/val', i)
  print(i, ' has ', len(os.listdir(dir_a)), ' images')

foot-ulcers  has  77  images
pressure ulcer  has  172  images
burns  has  92  images
venous  has  83  images
surgical  has  84  images


In [10]:
import tensorflow as tf
import urllib.request
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
pre_trained_model = VGG16(input_shape=(150,150,3),
                                    include_top=False,
                                    weights='imagenet')
pre_trained_model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [11]:
train_dir = '/content/Dataset_split/train'
val_dir = '/content/Dataset_split/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest',
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('block4_pool')
last_output = last_layer.output
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(5, activation='softmax')(x)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    ) # set as training data

validation_generator = train_datagen.flow_from_directory(
    val_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    )

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=20, restore_best_weights=True)
model_2 = Model(pre_trained_model.input, x)
model_2.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['acc'])

Found 2020 images belonging to 5 classes.
Found 508 images belonging to 5 classes.


In [12]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model_VGG16.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [13]:
history = model_2.fit(train_generator,
            steps_per_epoch= 2020/32,
            validation_steps= 508/32,
            epochs=100,
            validation_data=validation_generator,
            verbose=1,
            callbacks=[callbacks,mc]
            )

Epoch 1/100
64/63 [==============================] - ETA: 0s - loss: 18.0645 - acc: 0.4099
Epoch 1: val_acc improved from -inf to 0.55709, saving model to best_model_VGG16.h5
63/63 [==============================] - 43s 490ms/step - loss: 18.0645 - acc: 0.4099 - val_loss: 1.1701 - val_acc: 0.5571
Epoch 2/100
64/63 [==============================] - ETA: 0s - loss: 1.2791 - acc: 0.5010
Epoch 2: val_acc improved from 0.55709 to 0.57283, saving model to best_model_VGG16.h5
63/63 [==============================] - 28s 451ms/step - loss: 1.2791 - acc: 0.5010 - val_loss: 1.1579 - val_acc: 0.5728
Epoch 3/100
64/63 [==============================] - ETA: 0s - loss: 1.1846 - acc: 0.5594
Epoch 3: val_acc improved from 0.57283 to 0.63976, saving model to best_model_VGG16.h5
63/63 [==============================] - 28s 451ms/step - loss: 1.1846 - acc: 0.5594 - val_loss: 0.9741 - val_acc: 0.6398
Epoch 4/100
64/63 [==============================] - ETA: 0s - loss: 1.1218 - acc: 0.5807
Epoch 4: val_a